# Crear el dataset con la hipotéca de los clientes y guardarlo en un csv

## Vamos a importar las bibliotecas que usaremos

In [1]:
import requests
import pandas as pd
import requests
import json
import time
from tqdm import tqdm  # Para mostrar barra de progreso
pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.max_columns', None)
tqdm.pandas()  # Habilita barra para apply


## Función que devuelve una lista con la hipotéca del pensionado

In [ ]:
def cal_hipoteca(prop_price, ahorros_est,id_estado):
    time.sleep(5)
    ahorro_utilizado = min(ahorros_est, 0.15 * prop_price)

    url = "https://api.yave.mx/simulador/api/v2/simulations/"

    payload = json.dumps({
    "comprobacion_ingresos": 1,
    "contact": "no",
    "credit_type": "PURCHASE",
    "down_pmt": ahorro_utilizado,
    "hutk": "",
    "loan_requested": prop_price - ahorro_utilizado,
    "marital_status": 1,
    "page_url": "/simulador/credito-hipotecario/6?wf=true",
    "page_name": "Simula tu cr^édito hipotecario | Yave",
    "property_location": id_estado,
    "property_value": prop_price,
    "purchase_process_status": 4,
    "utm_campaign": "",
    "utm_content": "",
    "utm_medium": "",
    "utm_source": "",
    "utm_term": "",
    "ga_client_id": "1450756964.1746501294"
    })
    
    headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en,es;q=0.9,es-ES;q=0.8,en-GB;q=0.7,en-US;q=0.6,es-MX;q=0.5,es-US;q=0.4',
    'content-type': 'application/json',
    'origin': 'https://yave.mx',
    'priority': 'u=1, i',
    'referer': 'https://yave.mx/',
    'sec-ch-ua': '"Chromium";v="136", "Microsoft Edge";v="136", "Not.A/Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sl-origin': 'customers_web',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0'
    }


    r = requests.request("POST", url, headers=headers, data=payload)

    if r.status_code != 200:
        raise Exception(f"Error en la petición: {r.status_code} - {r.text}")

    data = json.loads(r.text)

    fijas = []

    for aux in data:
        if aux['payment_type'] == 'Fija':
            for aux_2 in aux['rates']:
                for aux_3 in aux_2['terms']:
                    fijas.append({
                        'cat': aux_3['cat'],
                        'mensualidad_prom': aux_3['payment'],
                        'ingreso_req': aux_3['required_income'],
                    })
    def promedio(lista, clave):
        return sum(d[clave] for d in lista) / len(lista) if lista else None

    return {
        'fijo_cat': promedio(fijas, 'cat'),
        'fijo_mensual': promedio(fijas, 'mensualidad_prom'),
        'fijo_ingreso': promedio(fijas, 'ingreso_req')
    }


## Función que me regresa el id del estado

In [ ]:
def id_estado(estado):
    estado_match_id = {
    # Estados aceptados directamente
    "SONORA": 26,
    "GUANAJUATO": 11,
    "SAN LUIS POTOSI": 24,
    "HIDALGO": 13,
    "SINALOA": 25,
    "MORELOS": 17,
    "JALISCO": 14,
    "DURANGO": 10,
    "GUERRERO": 12,
    "BAJA CALIFORNIA SUR": 3,
    "PUEBLA": 21,
    "NUEVO LEON": 19,
    "CHIHUAHUA": 8,
    "NAYARIT": 18,
    "BAJA CALIFORNIA": 2,
    "QUINTANA ROO": 23,
    "COAHUILA": 5,
    "CHIAPAS": 7,
    "YUCATAN": 31,
    "QUERETARO": 22,
    "AGUASCALIENTES": 1,

    # Estados no aceptados, mapeados a uno geográficamente cercano aceptado
    "SUR DE LA CDMX": 9,              # CDMX
    "ORIENTE DE LA CDMX": 9,
    "PONIENTE DE LA CDMX": 9,
    "NORTE DE LA CDMX": 9,
    "EL ESTADO DE MEXICO": 15,       # Estado de México
    "OAXACA": 21,                     # Puebla
    "VERACRUZ": 13,                   # Hidalgo
    "CAMPECHE": 7,                    # Chiapas
    "TLAXCALA": 13,                   # Hidalgo
    "TAMAULIPAS": 10,                 # Durango (o Nuevo León como alternativa)
    "TABASCO": 7,                     # Chiapas
    "ZACATECAS": 10,                  # Durango
    "COLIMA": 5,                      # Coahuila
    "MICHOACAN": 14                   # Jalisco
    }
    return estado_match_id[estado]

In [ ]:
def aplicar_hipotecas_por_fila(row):
    resultados = {}
    for q in ['q25', 'q50', 'q75']:
        prop_price = row[q]
        ahorros = row['ahorros_estimados']
        entidad_id = row['id_estado']
        
        try:
            result = cal_hipoteca(prop_price, ahorros, entidad_id)
            # Guardar en columnas distintas para cada quantil
            for key, val in result.items(): 
                resultados[f"{key}_{q}"] = val
        except Exception as e:
            # Si hay error (por ejemplo, en la petición), ponemos None
            for key in ['fijo_cat', 'fijo_mensual', 'fijo_ingreso']:
                resultados[f"{key}_{q}"] = None

    return pd.Series(resultados)

In [ ]:
df_antes = pd.read_csv('antes_del_97.csv')
df_despues = pd.read_csv('despues_del_97.csv')
precios = pd.read_csv('Precios_por_estado.csv')

df_antes['ahorros_estimados'] = df_antes['ultimo_sueldo']*12*4
df_despues['ahorros_estimados'] = df_despues['ultimo_sueldo']*12*4
df_antes['id_estado'] = df_antes['ent_pago_des'].apply(id_estado)
df_despues['id_estado'] = df_despues['ent_pago_des'].apply(id_estado)

df_antes = df_antes.merge(precios, left_on='ent_pago_des', right_on='estado', how='left')
df_despues = df_despues.merge(precios, left_on='ent_pago_des', right_on='estado', how='left')

In [ ]:
df_antes_muestra = df_antes.sample(n=10000, random_state=234)
df_despues_muestra = df_despues.sample(n=10000, random_state=10)
df_antes_muestra.name = 'antes_modelo'
df_despues_muestra.name = 'despues_modelo'

for df_aux in [df_antes_muestra, df_despues_muestra]:
    # Aplica a cada fila
    resultados_df = df_aux.progress_apply(aplicar_hipotecas_por_fila, axis=1)

    # Combina resultados con el DataFrame original
    df = pd.concat([df_aux, resultados_df], axis=1)
    df.to_csv(f"resultados_{df_aux.name}.csv", index=False)